In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [8]:
class BetsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_percentage: float
    summary: str

In [58]:
def calculate_sr(state: BetsmanState)->BetsmanState:
    sr=(state["runs"]/state["balls"])*100
    
    return {"sr":sr}

In [59]:
def calculate_bpb(state: BetsmanState)-> BetsmanState:
    bpb=state["balls"]/(state["fours"]+state["sixes"])
    
    return {"bpb": bpb}

In [60]:
def calculate_boundary_percentage(state: BetsmanState)-> BetsmanState:
    boundary_percentage=(((state['fours']* 4) + (state['sixes']*6))/state['runs'])*100
    
    return {"boundary_percentage": boundary_percentage}

In [61]:
def summary(state: BetsmanState)->BetsmanState:
    summary=f"""
    Strike Rate : {state['sr']}\n
    Balls per boundary : {state['bpb']}\n
    Boundary Percentage : {state['boundary_percentage']}\n
    """
    
    return {"summary": summary}

In [62]:
# crate graph
graph= StateGraph(BetsmanState)

graph.add_node("calculate_sr",calculate_sr)
graph.add_node("calculate_bpb",calculate_bpb)
graph.add_node("calculate_boundary_percentage",calculate_boundary_percentage)
graph.add_node("summary",summary)

# carete edges
graph.add_edge(START, "calculate_sr")
graph.add_edge(START, "calculate_bpb")
graph.add_edge(START, "calculate_boundary_percentage")

graph.add_edge("calculate_sr", "summary")
graph.add_edge("calculate_bpb", "summary")
graph.add_edge("calculate_boundary_percentage", "summary")
graph.add_edge("summary", END)

# Compile graph
workflow=graph.compile()



In [63]:
initial_state={
    "runs": 100,
    "balls": 50,
    "fours": 6,
    "sixes": 4 
}
final_state=workflow.invoke(initial_state)

In [64]:
print(final_state)

{'runs': 100, 'balls': 50, 'fours': 6, 'sixes': 4, 'sr': 200.0, 'bpb': 5.0, 'boundary_percentage': 48.0, 'summary': '\n    Strike Rate : 200.0\n\n    Balls per boundary : 5.0\n\n    Boundary Percentage : 48.0\n\n    '}
